<a href="https://colab.research.google.com/github/pandser/ai_assistant/blob/main/ai_assistant__my_arbitr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Практическая работа.**

**Создание неро-сотрудника на основе больших языковых моделей.**

В данной практической работе представлен ИИ-ассистент, косультирующий пользователя по вопросам подачи документов в Арбитражные суды РФ через систему "Мой арбитр".

### Установка необходимых библиотек.

In [ ]:
%%writefile requirements.txt
transformers>=4.42.0
llama_index
pyvis==0.3.2
Ipython==7.34.0
langchain==0.2.5
pypdf==4.2.0
langchain_community==0.2.5
llama-index-llms-huggingface==0.2.3
llama-index-embeddings-huggingface==0.2.2
llama-index-embeddings-langchain==0.1.2
langchain-huggingface==0.0.3
sentencepiece==0.1.99
accelerate==0.31.0
bitsandbytes
peft==0.11.1
llama-index-readers-wikipedia==0.1.4
wikipedia==1.4.0
llama-index-readers-file
gradio

arize-phoenix
gcsfs
nest-asyncio
openinference-instrumentation-llama-index
opentelemetry-api
opentelemetry-sdk
opentelemetry-exporter-otlp

# Зависимости
huggingface-hub==0.23.3
torch>=2.3.1
numpy==1.25.2
packaging==24.1
pyyaml==6.0.1
requests==2.31.0
tqdm==4.66.4
filelock==3.14.0
regex==2024.5.15
typing-extensions==4.12.2
safetensors==0.4.3
tokenizers==0.19.1

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

### Подготовка данных


Создадим базу знаний. Источником будет справочный раздел сайта ["Мой арбитр"](https://my.arbitr.ru/)

In [ ]:
!mkdir data # создание директории в которой будут собраны данные

In [ ]:
import re
import requests

from bs4 import BeautifulSoup


url = ("https://my.arbitr.ru/#help/4/0/")
html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')

In [ ]:
faq = soup.find_all('script', id=re.compile('^help*')) # поиск всех тегов <script>, id которых начинается на 'help'

In [ ]:
for num, data in enumerate(faq):
    raw = data.string.strip()
    if raw.startswith('<p') or raw.startswith('<h2'): # фильтруем теги в которых содержится интересующий нас текст
        text = re.sub(re.compile('<.*?>'), '', raw) # очищаем текст от тегов
        with open(file=f'data/{num}.txt', mode='w', encoding='utf-8') as f:
            f.writelines(text) # запись полученного текста в файл

### Подготовка модели

In [ ]:
import getpass
import os
import torch

from huggingface_hub import login
from langchain_huggingface  import HuggingFaceEmbeddings
from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
)
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.prompts import PromptTemplate
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.readers.file import PDFReader
from peft import PeftModel, PeftConfig
from pyvis.network import Network
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig,
)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.11/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
os.environ['HUGGINGFACE_ACCESS_TOKEN'] = getpass.getpass('Введите API Key:')
login(os.environ['HUGGINGFACE_ACCESS_TOKEN'])

Введите API Key:··········
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# Создание конфига, соответствующего методу PEFT (в нашем случае LoRA)
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,          # идентификатор модели
    quantization_config=quantization_config, # параметры квантования
    torch_dtype=torch.float16,               # тип данных
    device_map="auto"                        # автоматический выбор типа устройства
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
# Можно не переводить, но явное всегда лучше неявного
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [ ]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": 50,
      "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(
      model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    )
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Настройка ServiceContext (глобальная настройка параметров LLM)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

#### Подключение поисковика Auto-Merging Retriever из LlamaHub

In [ ]:
from llama_index.core.llama_pack import download_llama_pack

AutoMergingRetrieverPack = download_llama_pack(
    "AutoMergingRetrieverPack",
    "./auto_merging_retriever_pack",
)

In [ ]:
docs = SimpleDirectoryReader(
    "./data",
).load_data()

In [ ]:
auto_merging_pack = AutoMergingRetrieverPack(docs)

### Подключение Phoenix
Наблюдени за приложением LlamaIndex.

In [ ]:
import nest_asyncio
import phoenix as px

from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

In [ ]:
nest_asyncio.apply()  # необходим для параллельных вычислений в среде ноутбуков

In [ ]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://9a2wy6tzjke3-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Не удалось подключиться к Phoenix из-за ошибки 403 Forbidden.

### Запросы к ассистенту

In [ ]:
def get_response(query):
    message_template =f"""<s>system
        Ты ассистент для сайта 'Мой арбитр' https://my.arbitr.ru/.
        Отвечай в соответствии с Источником.
        Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
        Если нет, то просто скажи: 'я не знаю'. Не придумывай! </s>
        <s>user
        Вопрос: {query}
        Источник:
        </s>
        """
    return str(auto_merging_pack.run(message_template))

In [ ]:
query = 'Как подать документы от имени юридического лица?'
print(get_response(query))

Важно учесть, что подача документов от имени юридического лица может быть выполнена только представителем такого лица. В зависимости от формата подачи документов, представление может быть осуществлено на бумажной основе или онлайн через интернет.

Если документы подаются на бумажной основе, они должны быть подписаны представителем юридического лица и удостоверенные печатью. Кроме того, следует предоставить копию паспорта представителя и свидетельство о его полномочиях.

Если документы подаются онлайн, то представитель должен залогиниться в систему и выполнить процедуру подачи документов. Для этого необходимо заполнить все необходимые поля и предоставить необходимые документы в электронном виде.

После подачи документов, их можно проверить в режиме онлайн.


In [ ]:
query = 'Можно ли отправить документ без подписания электронной подписью?'
print(get_response(query))

> Merging 2 nodes into parent node.
> Parent node id: 8c025f6d-8b1a-4db6-b32d-0813a2e2b39a.
> Parent node text: Порядок подачи документов не предусматривает подачу электронных документов, подписанных присоедин...

Да, можно. В зависимости от того, на что вы хотите отправить документ, может быть разные требования. Например, для отправки документа в государственные органы, обычно требуется подписание электронной подписью. Но для отправки документа в частные организации или людей, это может быть необязательным.


### Вывод
Создан ИИ-ассистент на основе модели saiga_mistral_7b.

Подготовлена база знаний из открытого источника.

Для поиска документов использован пакет Auto-Merging Retriever из LlamaHub.

Не удалось подключиться к Phoenix из-за ошибки 403 Forbidden.
Для обеспечения безопасности планировал использовать meta-llama/LlamaGuard-7b, но запрос на использование модели был отклонен.
